In [1]:
scanners = {}
with open('Day19.txt') as file:
    for index, block in enumerate(file.read().split('\n\n')):
        block = block.split('\n')[1:]
        scanners[index] = frozenset(tuple(map(int, line.split(','))) for line in block) 

In [2]:
from functools import cache
from itertools import starmap, product

combinations = [
    lambda x, y, z: (+x, +y, +z),
    lambda x, y, z: (+x, +z, -y),
    lambda x, y, z: (+x, -y, -z),
    lambda x, y, z: (+x, -z, +y),
    lambda x, y, z: (-x, +y, -z),
    lambda x, y, z: (-x, +z, +y),
    lambda x, y, z: (-x, -y, +z),
    lambda x, y, z: (-x, -z, -y),
    lambda x, y, z: (+y, +x, -z),
    lambda x, y, z: (+y, +z, +x),
    lambda x, y, z: (+y, -x, +z),
    lambda x, y, z: (+y, -z, -x),
    lambda x, y, z: (-y, +x, +z),
    lambda x, y, z: (-y, +z, -x),
    lambda x, y, z: (-y, -x, -z),
    lambda x, y, z: (-y, -z, +x),
    lambda x, y, z: (+z, +x, +y),
    lambda x, y, z: (+z, +y, -x),
    lambda x, y, z: (+z, -x, -y),
    lambda x, y, z: (+z, -y, +x),
    lambda x, y, z: (-z, +x, -y),
    lambda x, y, z: (-z, +y, +x),
    lambda x, y, z: (-z, -x, +y),
    lambda x, y, z: (-z, -y, -x),
]

@cache
def all_combinations(scanner):
    return [frozenset(starmap(combination, scanner)) for combination in combinations]

In [3]:
def match(s1, s2):
    scan1, scan2 = scanners[s1], scanners[s2]
    for scan2 in all_combinations(scan2):
        for (x1, y1, z1), (x2, y2, z2) in product(scan1, scan2):
            position = dx, dy, dz = x1 - x2, y1 - y2, z1 - z2
            translations = frozenset((x + dx, y + dy, z + dz) for x, y, z in scan2)
            if len(scan1 & translations) < 12:
                continue
            return translations, position

In [4]:
%%time
scanner_positions = {0: (0, 0, 0)}
scanner_unmatched = set(range(1, len(scanners)))
todo = [0]
while todo:
    s1 = todo.pop()
    for s2 in list(scanner_unmatched):
        if result := match(s1, s2):
            translation, position = result
            todo.append(s2)
            scanner_unmatched.remove(s2)
            scanners[s2] = translation
            scanner_positions[s2] = position
len(frozenset.union(*scanners.values()))

CPU times: user 37.9 s, sys: 10.7 ms, total: 37.9 s
Wall time: 38 s


383

In [5]:
%%time
max_distance = 0
for (a, b, c), (x, y, z) in product(scanner_positions.values(), scanner_positions.values()):
    max_distance = max(max_distance, abs(a - x) + abs(b - y) + abs(c - z))
max_distance

CPU times: user 705 µs, sys: 0 ns, total: 705 µs
Wall time: 734 µs


9854